In [53]:
import pandas as pd
import numpy as np
import copy

In [61]:
# read in CPS data
cps = pd.read_csv('cps_2015_aug.csv')
# combine child tax credit and additonal child tax credit
cps['ctc_crd_total'] = cps.ctc_crd + cps.actc_crd
# list of all benefit variables
ben_vars = [('snap_impute', 'SNAP'), ('MedicaidX', 'Medicaid'), ('eit_cred', 'EITC'),
            ('f_mv_sl', 'school lunch'), ('hengval', 'energy assistance'),
            ('ctc_crd_total', 'child tax credit')]
# find total benefits
cps['tot_ben'] = 0
for var in ben_vars:
    cps['tot_ben'] += cps[var[0]]

In [62]:
# break down to families with a female reference person, no husband, 2 kids under 18
fam_cps = cps[(cps.fkind == 3) & (cps.fhusbidx == 0) & (cps.fownu18 == 2)]
print 'There are {} families that fit the criteria'.format(len(fam_cps))

There are 7809 families that fit the criteria


In [63]:
# narrow CPS down further to only those working 52 hours a wk, making $8/hour, and are 35 years old
per_cps = copy.deepcopy(fam_cps[(fam_cps.wkswork == 52) & (fam_cps.a_hrspay >= 8) & (fam_cps.a_age == 35)])
print 'There are {} people that fit the criteria'.format(len(per_cps))

There are 6 people that fit the criteria


In [64]:
def weighted_avg(sub_cps, var):
    vals = []
    weights = []
    for seq, seq2 in zip(sub_cps.h_seq, sub_cps.pf_seq):
        vals.append(cps[var][(cps.h_seq == seq) & (cps.pf_seq == seq2)].sum())
        weights.append(cps.fsup_wgt[(cps.h_seq == seq) & (cps.pf_seq == seq2)].sum())
    vals = np.array(vals)
    weights = np.array(weights)
    return (vals * weights).sum() / weights.sum()

In [65]:
# find weighted average of all benefits
ben_vars.append(('tot_ben', 'total'))
for var in ben_vars:
    print 'The average {} benefit is {}'.format(var[1], weighted_avg(per_cps, var[0]))

The average SNAP benefit is 2077.71277381
The average Medicaid benefit is 14326.5721988
The average EITC benefit is 2554.17697953
The average school lunch benefit is 1316.03118198
The average energy assistance benefit is 857.696299494
The average child tax credit benefit is 1880.87551396
The average total benefit is 23013.0649475
